In [20]:
import pandas as pd
import numpy as np
import json
import re
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

%matplotlib inline

In [2]:
labels = pd.read_csv("categories_string.csv")
train_label= pd.read_csv('train_label.csv')
with open('train.json') as f:
   train = json.load(f)
with open('test.json') as h:
   test = json.load(h)

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)
dic = labels.to_dict()
dic = dic["0"]
train_label['jobtitle'] = train_label['Category']
train_label = train_label.replace({"jobtitle": dic})
comb_data = pd.merge(train_df,train_label,how = "outer", on = 'Id' )
comb_data

,Id,description,gender,Category,jobtitle
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F,19,professor
1,1,He is a member of the AICPA and WICPA. Brent ...,M,9,accountant
2,2,Dr. Aster has held teaching and research posi...,M,19,professor
3,3,He runs a boutique design studio attending cl...,M,24,architect
4,4,"He focuses on cloud security, identity and ac...",M,24,architect
...,...,...,...,...,...
217192,217192,A member of the UWA Cultural Collections Boar...,M,19,professor
217193,217193,Kelly has worked globally leading teams of co...,F,22,psychologist
217194,217194,He's the lead author of a recent study that f...,M,19,professor
217195,217195,She specializes in the theoretical and pedago...,F,19,professor


In [3]:
sentences = comb_data['description'].tolist()
len(sentences)
sentences_as_one_string =" ".join(sentences)

In [4]:
comb_data['description'].apply(lambda x: len(x.split(' '))).sum()

13504784

In [6]:
def print_train(index):
    example =comb_data[comb_data.index == index][['description', 'Id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

In [7]:
print_train(10)

 An elder in the United Methodist Church, she writes and lectures widely about African-American religion and is treasurer of the Society for the Study of Black Religion. She is an expert on women in the civil rights movement, and she wrote Witnessing and Testifying: Black Women, Religion, and Civil Rights (Augsburg Fortress, 2003). Contact 404-270-5527, RRoss@spelman.edu.
Tag: 10


In [8]:
sentences_test = test_df['description'].tolist()
len(sentences_test)
sentences_test_as_one_string =" ".join(sentences)

In [9]:
def print_test(Id):
    example =test_df[test_df.Id == Id][['description', 'Id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

In [10]:
print_test(84)

 He is coauthor with Robert N. Brandon of Biology’s First Law: The Tendency for Diversity and Complexity to Increase in Evolutionary Systems (University of Chicago Press, 2010) and is coauthor with Alex Rosenberg of Philosophy of Biology: A Contemporary Introduction (Routledge, 2008).» Post Comment
Tag: 84


In [12]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [13]:
%%time
comb_data['description'] = comb_data['description'].apply(clean_text)

Wall time: 34.1 s


In [14]:
print_train(10)

elder united methodist church writes lectures widely africanamerican religion treasurer society study black religion expert women civil rights movement wrote witnessing testifying black women religion civil rights augsburg fortress 2003 contact 4042705527 rross spelmanedu
Tag: 10


In [15]:
%%time
test_df['description'] = test_df['description'].apply(clean_text)

Wall time: 8.39 s


In [16]:
print_test(84)

coauthor robert n brandon biologys first law tendency diversity complexity increase evolutionary systems university chicago press 2010 coauthor alex rosenberg philosophy biology contemporary introduction routledge 2008 post comment
Tag: 84


In [17]:
X_test = test_df.description
X = comb_data.description
y = comb_data.Category

print(X.shape)
print(y.shape)
print(X_test.shape)

(217197,)
(217197,)
(54300,)


In [18]:
X_train, X_valid, y_train ,y_valid = train_test_split(X, y, test_size=0.25, random_state = 1)

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(162897,)
(162897,)
(54300,)
(54300,)


In [24]:
svm_rbf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(kernel='rbf',C= 1.0, gamma = 0.1)),
                     ])

In [25]:
svm_rbf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,

In [28]:
predicted = svm_rbf.predict(X_valid)

In [29]:
print(metrics.classification_report(y_valid, predicted))

              precision    recall  f1-score   support

           0       0.62      0.37      0.46       361
           1       0.92      0.60      0.73      1015
           2       0.71      0.71      0.71       248
           3       0.55      0.50      0.52      2285
           4       0.82      0.50      0.62       194
           5       0.81      0.71      0.76      1117
           6       0.56      0.75      0.64      3114
           7       0.85      0.39      0.53       211
           8       0.85      0.66      0.74      1656
           9       0.88      0.58      0.70       771
          10       0.91      0.36      0.52       214
          11       0.72      0.68      0.70      2891
          12       0.88      0.57      0.69       421
          13       0.73      0.57      0.64      1014
          14       0.94      0.74      0.82      3133
          15       0.76      0.64      0.70      1148
          16       0.93      0.87      0.90      1326
          17       0.93    

In [30]:
print(metrics.f1_score(y_valid, predicted,average = 'macro'))

0.6810253212823848


In [31]:
prediction = svm.predict(X_test)

AttributeError: module 'sklearn.svm' has no attribute 'predict'

In [ ]:
prediction

In [ ]:
test_df["Category"] = prediction
baseline_file = test_df[["Id","Category"]]
baseline_file.to_csv("baseline_svm_rbf.csv", index=False)